<a href="https://colab.research.google.com/github/omidshm/tradingview-sentiment/blob/cooking-jupyter-mvp/tradingview_sentiment_mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install requests-HTML

In [5]:
import requests
from requests_html import HTMLSession

In [6]:
import json

In [23]:
import pandas as pd

In [74]:
import time

In [7]:
url = 'https://www.tradingview.com/markets/cryptocurrencies/ideas/'



In [71]:

# crawl function

def get_page(url: str) -> HTMLSession:
  try:
    session = HTMLSession()
    response = session.get(url)

    return response
     
  except requests.exceptions.RequestException as e:
    print(e)

In [72]:
response = get_page(url)

In [9]:
widget_xpath = '//*[@id="js-category-content"]/div/div/div/div/div/div/div[2]/div/div'
tv_widgets = response.html.xpath(widget_xpath)

In [69]:
idea_item_sample = {
    'symbol':'BTCUSDT',
    'default_sentiment':'LONG',
    'timeframe':'60',
    'title':'BTC LOOKS BULLISH',
    'link':'https://tgview.com/sdsfsdfsfsdf',
    'timestamp':'date'
}

idea_items = []

In [30]:
# get page items and return idea_items
def get_item(widget):
  idea_item = {}

  symbol_xpath = '//div[2]/div/a'
  sentiment_xpath = '//span[3]/span[2]'
  timeframe_xpath = '//div[2]/span[2]'
  title_xpath = '//div[1]/a'
  link_xpath = '//div[1]/a'
  date_path = '//div[4]/span[3]/span/span'
  BASE_TGVIEW_URL = 'https://www.tradingview.com'

  try:
    idea_item['symbol'] = widget.xpath(symbol_xpath)[0].text
    
  except:
    print('Logging: some adds blocked')
  else:
    try:
      idea_item['default_sentiment'] = widget.xpath(sentiment_xpath)[0].text
    except:
      idea_item['default_sentiment'] = ''
    
    idea_item['timeframe'] = widget.xpath(timeframe_xpath)[0].text
    idea_item['title'] = widget.xpath(title_xpath)[0].text
    idea_item['timestamp'] = widget.xpath(date_path)[0].attrs['data-timestamp']
    
    idea_item['link'] = BASE_TGVIEW_URL+widget.xpath(link_xpath)[0].attrs['href']

    idea_items.append(idea_item)

    print(f'{idea_item} added to items list')

  
  

In [ ]:
for widget in tv_widgets:
  get_item(widget)

In [32]:
items_df = pd.DataFrame(idea_items)

In [39]:
items_df.to_csv('tgview_ideas.csv', sep=';')

In [54]:
pages_number_object = response.html.xpath('//*[@id="js-category-content"]/div/div/div/div/div/div/div[3]/div/div/div[2]/a[3]')[0]
pages_number_str = pages_number_object.text
pages_number_int = int(pages_number_str)

In [63]:
# generate pagination Links
def last_page_finder(pages_number_object):
  pages_number_str = pages_number_object.text
  pages_number_int = int(pages_number_str)

  return pages_number_int

In [80]:
# pagination between all pages and save them to the df
def pagination_crawl(response) -> list:
  pages_number_object = response.html.xpath('//*[@id="js-category-content"]/div/div/div/div/div/div/div[3]/div/div/div[2]/a[3]')[0]

  last_page_num = last_page_finder(pages_number_object)

  BASE_PAGE_URL = 'https://www.tradingview.com/markets/cryptocurrencies/ideas/'
  for page in range(last_page_num):
    page_number = page+1

    if page_number == 1:
      response = get_page(BASE_PAGE_URL)

      print()

      widget_xpath = '//*[@id="js-category-content"]/div/div/div/div/div/div/div[2]/div/div'
      tv_widgets = response.html.xpath(widget_xpath)

      for widget in tv_widgets: 
        get_item(widget)
    else:
      request_url = BASE_PAGE_URL+'page-'+str(page_number)+'/'

      response = get_page(request_url)

      widget_xpath = '//*[@id="js-category-content"]/div/div/div/div/div/div/div[2]/div/div'
      tv_widgets = response.html.xpath(widget_xpath)

      for widget in tv_widgets: 
        get_item(widget)
    
    time.sleep(7)

      

In [81]:
pagination_crawl(response)


{'symbol': 'BTCUSDT', 'default_sentiment': '', 'timeframe': '60', 'title': 'BITCOIN - Update', 'timestamp': '1645025200.0', 'link': 'https://www.tradingview.com/chart/BTCUSDT/6wEWLyO4-BITCOIN-Update/'} added to items list
{'symbol': 'SHIBUSDT', 'default_sentiment': 'Long', 'timeframe': '240', 'title': 'SHIBA can go to 4000 sat this week and give around 30% profit', 'timestamp': '1645008095.0', 'link': 'https://www.tradingview.com/chart/SHIBUSDT/TgNfRwRY-SHIBA-can-go-to-4000-sat-this-week-and-give-around-30-profit/'} added to items list
{'symbol': 'BTCUSD', 'default_sentiment': '', 'timeframe': '1D', 'title': 'BTC Bitcoin: Predicting The Future', 'timestamp': '1645046536.0', 'link': 'https://www.tradingview.com/chart/BTCUSD/3ZvudNDw-BTC-Bitcoin-Predicting-The-Future/'} added to items list
{'symbol': 'BTCUSDT', 'default_sentiment': 'Long', 'timeframe': '60', 'title': 'BITCOIN IS STILL BULLISH AND CAN GO TO 45.000-47.000 VERY SOON', 'timestamp': '1645007292.0', 'link': 'https://www.tradi

In [82]:
len(idea_items)

1000

In [79]:
idea_items

[]

In [83]:
items_df = pd.DataFrame(idea_items)
items_df.to_csv('tgview_ideas_1000.csv', sep=';')